In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

def rough_plot(in_list, num_group):
    rough_list = list()
    tmp_sum = 0
    
    for i in range(1, len(in_list)):
        tmp_sum += in_list[i]
    
        if(i%num_group == num_group-1):
            rough_list.append(tmp_sum / num_group)
            tmp_sum = 0
            
    plt.plot(rough_list)

In [3]:
import math
from fractions import Fraction

class StockQLData:
    'Result data of approximated q-learning of stock price'
    
    def __init__(self, action_level, data_len, ratio_num_local, trial_range):
        self.last_reward = list()
        
        self.last_w = list()
        
        self.rwd_sum = 0
        self.avg_point_rwd = list()

        self.act_num = np.zeros(action_level*2 + 1)
        self.act_num_list = list()

        self.ratio_local_size = math.floor(data_len/ratio_num_local)
        self.tmp_local_act_num = np.zeros(action_level*2 + 1)
        self.tmp_trial_act_ratio = list()
        self.act_ratio = list() # all_trial_act_ratio
        
        self.trial_range = trial_range # for the default argumant of print_local_act_ratio
        
    def point_update_action_num(self, last_act):
        self.act_num[last_act] += 1

    
    def point_update_rwd_sum(self, state, last_state, avg_point_rwd_func):
        self.rwd_sum += avg_point_rwd_func(state, last_state) 
        
    def point_update_action_ratio(self, last_act, data_index):        
        self.tmp_local_act_num[last_act] += 1
        if (data_index+1) % self.ratio_local_size == 0:
            self.tmp_trial_act_ratio.append(self.tmp_local_act_num/self.tmp_local_act_num.sum())
            self.tmp_local_act_num.fill(0)
            
        
    def last_point(self, trial_length, state, init_invest, w, last_rwd_func):
        self.avg_point_rwd.append(self.rwd_sum/(trial_length-1)) #ignore the first reward(=0)
        self.rwd_sum = 0
        
        self.last_reward.append(last_rwd_func(state, init_invest))
        
        cp_w = w.copy()
        self.last_w.append(cp_w)

        tmp_act_num = self.act_num.copy()
        self.act_num_list.append(tmp_act_num)
        self.act_num.fill(0)
        
        if np.count_nonzero(self.tmp_local_act_num) > 0:
            self.tmp_trial_act_ratio.append(self.tmp_local_act_num/self.tmp_local_act_num.sum())
            self.tmp_local_act_num.fill(0)
        self.act_ratio.append(self.tmp_trial_act_ratio.copy())
        del self.tmp_trial_act_ratio[:] # empty the list

        
    def __act_ratio_prepare_print(self, start, end, alcnum, action_level): # the start/end trial index(can not recognize in the plot, just where data come from), actual number of local(i.e. group)
        ret = list()
        
        # initialize
        for i in range(0, alcnum):  # alcnum
            ret.append(np.zeros(action_level*2 + 1))
            
        trind = start
        lcind = 0
        for trind in range(start, end+1):
            for lcind in range(0, alcnum):
                for actind in range(0, action_level*2 + 1):
                    ret[lcind][actind] += sqld.act_ratio[trind][lcind][actind] / (end - start + 1)
        return ret

        
    # print the series with local rate ratio
    def print_local_act_ratio(self, data, action_level, start_trind=0, end_trind=None):
        if end_trind is None:
            end_trind = trial_range - 1
        
        alcnum = len(sqld.act_ratio[0]) # actual number of local(i.e. group)
        
        to_plot = self.__act_ratio_prepare_print(start_trind, end_trind, alcnum, action_level)
        
        sf, saxarr = plt.subplots(alcnum, 2, figsize=(9, 4*alcnum)) # print all local by now
        
        row_count = 0
        
        pie_labels = ["hold"]
        for i in range(1, action_level+1):
            pie_labels.append("{0} buy".format(Fraction(i, action_level)))
        for i in range(1, action_level+1):
            pie_labels.append("{0} sell".format(Fraction(i, action_level)))
            
        for lcind in range(0, alcnum): # print all local by now
            tmp_data = data.copy()
            tmp_data = tmp_data[sqld.ratio_local_size*lcind : sqld.ratio_local_size*(lcind + 1)]
            
            saxarr[row_count, 0].plot(tmp_data)
            saxarr[row_count, 0].set_title('close_price')
            
            saxarr[row_count, 1].pie(to_plot[lcind], labels=pie_labels)
            saxarr[row_count, 1].set_title('act_ratio')
            
            
            row_count += 1
        

after an action ---> call it the (new) state  
here: last_state_bf_act -> |act| -> last_state -> |new price release| -> (new) state  
but the last_state can be dismiss because when calculating(my_reward), it can be alternated by last_state_bf_act and keep the rule of the first line(i.e. after an action ---> call it the (new) state)

In [4]:
conslist = list()
consrange = 20
for i in range(0, consrange*2 + 1):
    conslist.append([0, 0])
    
cons = 0


relist = ['QCOM.csv', 'GOOG.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', '^N225.csv', '^GDAXI.csv', \
          '^IXIC.csv', '^KS11.csv', '^SSEC.csv', \
         'NSU.DE_10_5_5.csv', 'DAI.DE_10_5_5.csv', 'BMW.DE_10_5_5.csv']
# for data in input/2001_2010/
relist = ['QCOM.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', '^N225.csv', '^GDAXI.csv', \
          '^IXIC.csv', '^KS11.csv', '^SSEC.csv', \
         'NSU.DE.csv', 'DAI.DE.csv', 'BMW.DE.csv']

#relist = ['QCOM.csv']
#relist = ['DAI.DE_10_5_5.csv']
for resource_data in relist:
    
    for i in range(0, consrange*2 + 1):
        conslist[i][0] = 0
        conslist[i][1] = 0
    
    #input_csv = "./input/{0}".format(resource_data)
    input_csv = "./input/2001_2010/{0}".format(resource_data)
    qcom = pd.read_csv(input_csv)

    qclose = qcom['Close']

    qclose = qclose.fillna(qclose.median())

    qclose = qclose.tolist()

    np.isnan(qclose).any()
    


    init_invest = 10000
    
    trial_range = 10
    
    for trial in range(0, trial_range):
        #print("trial = ", trial)

        state = {'NumStock': 0, 'StockPrice': qclose[0], 'HandCash': init_invest}
        last_state = {'NumStock': 0, 'StockPrice': qclose[0], 'HandCash': init_invest}
        last_state_bf_act = {'NumStock': 0, 'StockPrice': qclose[0], 'HandCash': init_invest}
        
        cn = 0 # consecutive day num


        for point in range(0, len(qclose)):
            
            state['StockPrice'] = qclose[point]
            
            if point == 0:
                cn = 1 # serve as stok price going up
            elif 1 <= cn <= consrange: # consequtive goes up
                if qclose[point] >= qclose[point - 1]:
                    conslist[cn][0] += 1
                    cn += 1
                else:
                    conslist[cn][1] += 1
                    cn = consrange + 1
            elif 21 <= cn <= consrange*2: # consequtive goes down
                if qclose[point] < qclose[point - 1]:
                    conslist[cn][1] += 1
                    cn += 1
                else:
                    conslist[cn][0] += 1
                    cn = 1
            
            tmp = cn
            
            # ar: action ratio
            while(1):          
                if conslist[tmp][0] + conslist[tmp][1] > 0:
                    ar = conslist[tmp][0] / (conslist[tmp][0] + conslist[tmp][1]) # ar: prob of next price going up
                    break
                else:
                    tmp -= 1
                    if tmp == 0 or tmp == consrange:
                        ar = 0.5
                        break
                    
            if ar >= 0.5: # next price may go up
                #buy
                ar = (ar - 0.5)*2
                #state['NumStock'] += state['HandCash'] * ar / state['StockPrice']
                #state['HandCash'] *= 1 - ar
                state['NumStock'] += state['HandCash'] / state['StockPrice']
                state['HandCash'] = 0
                
                
            else: # next price may go down
                #sell
                ar = 1 - ar # ar: prob of next price going down
                ar = (ar - 0.5)*2
                #state['HandCash'] += state['NumStock'] * ar * state['StockPrice']
                #state['NumStock'] *= 1 - ar
                state['HandCash'] += state['NumStock'] * state['StockPrice']
                state['NumStock'] = 0
                
                
            #print(state)
            #print(point, ' :: ', conslist[1:15], '*****', conslist[consrange+1: consrange+15])
    print(resource_data, state['HandCash'] + state['StockPrice']*state['NumStock'])
    #print(resource_data, state['HandCash'] + state['StockPrice']*state['NumStock'])
    #print(point, ' :: ', conslist[1:6], '*****', conslist[consrange+1: consrange+6])

    #rough_plot(sqld.last_reward, 2)

QCOM.csv 54828.08636441114
NOK.csv 3287.612143102513
^DJI.csv 36111.572942170795
^TWII.csv 45351.861510030074
^N225.csv 61885.25081884552
^GDAXI.csv 58197.61049152301
^IXIC.csv 12695.766052948367
^KS11.csv 21951777.099110607
^SSEC.csv 27100.48396271439
NSU.DE.csv 102805.80392896173
DAI.DE.csv 15752.83180141713
BMW.DE.csv 33844.732677163425


#### conslist explanation
* **conslist[1] = [605, 332] ----> stock rose (one day in a row) include today**, in our previous experience, there were 605 days the tomorrow price raise, 332 days the tomorrow price fall
* conslist[2] ----> stock rose two days in a row include today
* ...
* conslist[consrange] ----> stock rose {consrange} days in a row include today
* conslist[consrange + 1] ----> stock fell (one day in a row) include today
* conslist[consrange + 2] ----> stock fell two days in a row include today
* ...
* conslist[consrange*2] ----> stock fell {consrange} days in a row include today
* **conslist[3] = [605, 332] ----> stock rose 3 days in a row include today. In our previous experience, there were 605 days the tomorrow price raise, 332 days the tomorrow price fall in the same situation(i.e. rose 3 days in a row).**

In [5]:
i

40

In [6]:
point

2575

In [7]:
trial

9

In [8]:
state

{'HandCash': 33844.732677163425, 'NumStock': 0, 'StockPrice': 58.849998}

In [9]:
#conslist

In [10]:
resource_data

'BMW.DE.csv'

In [11]:
conslist

[[0, 0],
 [3500, 3000],
 [1680, 1810],
 [860, 820],
 [460, 400],
 [190, 270],
 [70, 120],
 [30, 40],
 [20, 10],
 [10, 10],
 [10, 0],
 [0, 10],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [3320, 3170],
 [1650, 1520],
 [830, 690],
 [340, 350],
 [230, 120],
 [90, 30],
 [10, 20],
 [0, 20],
 [10, 10],
 [0, 10],
 [10, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]